In [268]:
# Dependency is dataset2_crowdtruth_working.csv

import pandas as pd
import numpy as np
import string
import re
from w266_common import utils, vocabulary
from tensorflow.contrib.learn import preprocessing
from itertools import chain
from nltk.corpus import wordnet
from nltk.corpus import brown
from nltk.corpus import reuters
from collections import defaultdict

pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)


In [269]:
# Change dataset2_crowdtruth_working.csv with correct path
df = pd.read_csv('./Data/dataset2_crowdtruth_working.csv',encoding='latin-1')
df_funny = df[df['funny']==1]
df_not_funny = df[df['not_funny']==1]
data = df_funny['text']
df.head()

,text,funny,not_funny
0,"left luggage , also luggage storage or bag storage , is a place where one can temporarily store one 's luggage so as to not have to carry it.",0,1
1,"their name comes from the title of the the flaming lips album , hit to death in the future head.",0,1
2,the lyside sulphur can grow from egg to adult in as little as 13 days.,0,1
3,istanbul suicide bomber entered turkey as a migrant: turkish pm,0,1
4,"both describe god as the one divine person , jesus christ , who has a divine soul of love , divine mind of truth , and divine body of activity.",0,1


In [270]:
# tokenizer = RegexpTokenizer(r'\w+')
# tokenized_train = [tokenizer.tokenize(x) for x in train_data]
# EDA on string length
def word_count(string):
    tokens = string.split()
    n_tokens = len(tokens)
    return n_tokens 

wordcounts = data.apply(word_count)
max_doc_len = max(wordcounts)
print(max_doc_len)

# processor = preprocessing.VocabularyProcessor()


157


In [271]:
# Create vocab processor object and fit data
processor = preprocessing.VocabularyProcessor(max_doc_len)
processor.fit_transform(data)


<generator object VocabularyProcessor.transform at 0x1381b9468>

In [272]:
# Build vocab list
vocab_dict = processor.vocabulary_._mapping
sorted_vocab = sorted(vocab_dict.items(), key = lambda x : x[1])
vocabulary_list = list(list(zip(*sorted_vocab))[0])
vocabulary_list[0:5]
len(vocabulary_list)

9097

In [273]:
# Get synonyms for all words in vocab
def synonym(text):
    synonyms = wordnet.synsets(text)
    syn_list = chain.from_iterable([word.lemma_names() for word in synonyms])
    return set([x.lower() for x in syn_list])

syn_set = [synonym(x) for x in vocabulary_list]
word_syn_dict = dict(zip(vocabulary_list,syn_set))

In [274]:
# Create combined brown + reuters corpus to get frequencies of words in common semantic usage
words = brown.words() + reuters.words()
words = map(lambda x:x.lower(),words)

freqs = defaultdict(int)
for word in words:
    freqs[word] += 1
    
print("There are",str(len(freqs)),"unique words")

There are 66519 unique words


In [275]:
freqs

defaultdict(int,
            {'the': 139248,
             'fulton': 17,
             'county': 252,
             'grand': 67,
             'jury': 73,
             'said': 27344,
             'friday': 375,
             'an': 8297,
             'investigation': 104,
             'of': 73191,
             "atlanta's": 4,
             'recent': 770,
             'primary': 252,
             'election': 133,
             'produced': 288,
             '``': 8837,
             'no': 3634,
             'evidence': 267,
             "''": 8789,
             'that': 18134,
             'any': 2269,
             'irregularities': 9,
             'took': 566,
             'place': 760,
             '.': 144033,
             'further': 1105,
             'in': 50590,
             'term-end': 1,
             'presentments': 1,
             'city': 618,
             'executive': 393,
             'committee': 622,
             ',': 130694,
             'which': 7227,
             'had': 8108,
     

In [276]:
# Get the most common synonym
for key, value in word_syn_dict.items():
    dictionary = dict([[word,freqs[word]] for word in value])
    try:
        word_syn_dict[key] = max(dictionary, key = dictionary.get)
    except:
        word_syn_dict[key] = key

In [277]:
# Print out some examples
word_syn_dict

{'<UNK>': '<UNK>',
 'donâ': 'donâ',
 't': 't',
 'knock': 'belt',
 'on': 'on',
 'deathâ': 'deathâ',
 's': 's',
 'door': 'door',
 'hit': 'make',
 'the': 'the',
 'doorbell': 'bell',
 'and': 'and',
 'run': 'work',
 'he': 'he',
 'hates': 'hat',
 'that': 'that',
 'i': '1',
 'discovered': 'see',
 'scream': 'call',
 'same': 'like',
 'way': 'way',
 'whether': 'whether',
 'iâ': 'iâ',
 'm': 'k',
 'about': 'about',
 'to': 'to',
 'be': 'be',
 'devoured': 'down',
 'by': 'by',
 'a': 'a',
 'great': 'capital',
 'white': 'white',
 'shark': 'shark',
 'or': 'or',
 'if': 'if',
 'piece': 'while',
 'of': 'of',
 'seaweed': 'seaweed',
 'touches': 'meet',
 'my': 'my',
 'foot': 'base',
 'wife': 'wife',
 'used': 'use',
 'regular': 'even',
 'customer': 'customer',
 'at': 'at',
 'mcdonalds': 'mcdonalds',
 'these': 'these',
 'days': 'years',
 "she's": "she's",
 'more': 'more',
 'large': 'great',
 'two': '2',
 'cans': 'can',
 'paint': 'key',
 'got': 'have',
 'married': 'married',
 'later': 'after',
 'bride': 'bride',

In [218]:
def word_replacer(sentence):
#     word_list = sentence.split()
#     punct = []
#     revised_sent = []
    
#     for word in sentence.split():
#         punct = re.sub(r'[\w\s]','',word)
#         try:
#             revised_sent.append(word_syn_dict[re.sub(r'[^\w\s]','',word)]+punct)
#         except:
#             revised_sent.append(word)      
#     return " ".join(revised_sent)


In [98]:
# apply synonym function to dataframe
# df['synonym_text'] = df['text'].apply(word_replacer)










































































,








'




























































































































































































'























































'





'














































































































































,



























































































































































































































































































'











,







'
















































()

,

















,








,





?



















































(

)

,






































'




'



















,





















.






































,











:
















:








-

..





'




-













,


















,























-
..
,



,


































..




,













'












..














'







-:






:
:







.

,

-




,


:



























:
..

'








:


..








:

..






'


















.




:









:






,





,











-







''













:






.






,
:






;




















:









'
-':





''
..

'



-














.
'



:






,






'
'

'










,








-








'--'


'
:







.







:






,



..





:









-





-
:























:







'':






.
,




?
'

:








































:

:

,


,


,







.










.







''













:





'

':





'



















:










.
-



:








,




:













'

































.






..




:









'
':



'




:







-

$


:
..






,








'
'

:



-

























'



'
:







'
:









'

':






:




















:



?







































;




















-





,


:

"




,"





.

:








:














:




,








:


'
'




:
..










:













,









..











:



;
-


:
'





,

:
:








,

.








,



´
:


,



,

















--
:
,








:









,




,


'


,
..











'











..



'








































'





:


,



:



'
























,









'






'














'












..













,







-






:











:



,





:

,










''

..








:








:
'



-





,




:
-






:
:




'
,




:


-





,


-


,




,

:


'





'













:







,


















:





;
















,
,



















'



















-
'







-





'




'





'
'









''

:

..'












..


'
'




,

:


..

















:







:




















:
..


























..

''






,







































:

,




























,













;








'
.




:

'







'

'

'

,


'

'

'




'









'




,

:

'













$.





..











































''

-






,











'

:


..,



-




,
















-









..
,






















,









'





,





'---'
:



















..
















:




























'

'




"
"

:




















;


:







''
















$.









','





















,
-

























:
















'







-





-'










'



:











-


























,












..,






:
..

..






'

'









'










'









'










..


























:


'


,








,


:







-




























:







'
'




:
..









:



'
'

..



-
:













,




:

..


-


:



'

'
'









,




-
:

..










:
..,






:
..







..



?




:


,
'
?





,



:








:







,

:
























,




































''








,


,



,





















:




'
'







..












:




..



:























:















:





;


















:









.














:






''

:



,
'




:









:


















'
:




'





:















,









,














-


..


'










:

'
'
'






























'



:





'


,



.
.









'
:




-



'
:












,

,


:




"


,





":








:
..




:




:


-












..,


'




,






:

'
















:
'



:















''

,








-

/-





,


:










:


,-
,






:















'



..,








:




..

















:

-








:











:





,












,
:





:
'



...

?
:
,


























.














'

'
.






'














.






..
'



.











?










.













.



;





.





















.
'












.

'





...






.





'
'

!!'



.


?


,





.


'














.


'

,


,





'

,


?







?!
'
!










!



$.









!






.

















.



'



















.



?




!
'





,
.



















.












?














'



.



.



.







.


""

.


.
'
















''.

,
.















?





,






.
:






.
















?
'




.
'

'





.




?




?

'

,









,
'

.



















.






.




&;

:







'


?
:
,
!
.



?
.


.
:
,



'



.
:

.








,

.









,


"


,
"
:












.





















.





.



.









.

.















?
'






.


-




,





?



.













,
.








:





,

'



'















,
'











'






'


,










.








.





,



.








?



.
-:



'



,
-
,






.
:










,








,
:


















.
:



,


,


.











.


:


,











'
,
















.
:



,













.
:
()


.
:







.
:











.
:





.

:








.
:


""

'

.
:









.
:
'





.
:






,








-
:






:






'






,
'


.

:








'



.
:





'







.
:












.
:











.
",









."



.


',
'


.







?











?











.



.




.




--




.
,

,










.












"

,
"





.








.




,







.






,




.


...
'



.





.





.








.





.
,
,

,




...

'




.


'


'




.




:

.
:




.
:

.

-









.










!
:

-






.
"!"
:





.
'






.





,

,




.







,



.




.











.




.








¦





,


.

















.


'

.

'

.













...








.















?





.









.












.


'



'




.

:



.






?
'

!









.





.\

,
'
.







.











:











?
:




???





.




.





,







.
















.





,









.













.












"





."





,


...





,


,



,
"





?"

:
"


?"








.







,















.








,








.






.



"





:!"

:
"?



:?"
!
!
'
?
.

?

.
!
!
'
?
.

?





.


,


.









.



'

.






.










.
'




-





.
'














.

'



,






,


.

'




,










.




,








.
'

,

'









'
.
'






-
'






.
#



,
'





'
.






.
























.



.

,
'





.




..




.





,


;





,

.





.

























.





,

.










,








.






.
?





.




,

,
'




.



,


,



.


,







.

.
%



.
:



'





.












.



"-"








,

,




.










.





""
,
'
?
'
""


","
'


!
-





;
'








.












.









,
'

.

'



?

'



.

...,
'
.

'


.






'





.



,
'


.
'




'






.





.






.







,








.



,



.




.




,


%

.










,







.








.
'

'


.
'




.
















?






?










?










?




.









'




.







.



'


.















.




",

'


,
."





,

"

'

.
'
."

,
",


!"

,
"


!"






.



,
'





.






'





.











.








?


.






.



±









?



















.


.





?

£












?
'



'
.





.
'











'



©.













?





.






?

'







'



?
'





.








?





.




?








.










.









.

'



?






-.













?














?


.







?








.



'


.














?
.







.

















.



-.
'






.


'






.










'
.














'
.












,







.
'







'




...







.







?












.
'






.
'




.
'







?









.










?

'.






?


.







.






'





-.











....




.







.


..

?






.











.


.










.












:
"






!"










?









?








.






.





?



.

'




?



.










?





,

?







?






.

'


'
?

'


..









?

.









?

'




.






?






.






?




.






?


.

'


?




.







?
,




.





.










$




.

,

$?

:
"'



,
'


."




?



.





?


±
.






.











.









?



.





?




"



",




.





?












.







?



.
'







?







.





?





.








?
'



.







?






















?

-


.




?




.








?

,

.






?










?

.










?

'

,

'
!



.





.





?

-.






?










?
'


.










?

.







?



.








?


'





?



















?
,
.












?





.

'


?






.






?




-.








?





.










?

-.







..



.








?



-.








?



'
-!







?






,


'






.












?
'




.











?



,



.










?


'





.









?



.





?

.










?


'







-
?


.





?
.






,
,
,
,





?

!







.



.



.








,






In [278]:
def least_common_replace(sentence):
    dictionary = {word:freqs[re.sub(r'[^\w\s]','',word)] for word in sentence.split() if len(word) >= 4}
    try:
        least_common = min(dictionary, key = dictionary.get)
        print(least_common)
        revised_sent = []
        for word in sentence.split():
            punct = re.sub(r'[\w\s]','',word)
            if re.sub(r'[^\w\s]','',word) == least_common:
                try:
                    revised_sent.append(word_syn_dict[re.sub(r'[^\w\s]','',word)]+punct)
                except:
                    revised_sent.append(word)
            else:
                revised_sent.append(word)
        return " ".join(revised_sent)
    except:
        return sentence

In [279]:
df_funny['revised_text'] = df_funny['text'].apply(least_common_replace)

donât
iâm
she's
cans
irreplaceable,
she's
guys
can't
polack
roses
what's
couldn't
isn't
toaster.
nonsense!
"sunny-side
disproved
apologise'
yo'--a
carries
obedient
pharmacist.
redundant.
you'll
"lazy."
this?""ever
i've
snore
hugo
tasks?
nastics
amin
saying.â
wrap
wrap
lol"
liar,
maths"
landlord
quit
"there'd
comic
sit-ups"
sniper"
percy
"who's
-corinna
you've
doesn't
archaeologist.
for"."oh
can't
how's
they're
mcraken
he's
"socrates
you're
highway
isnât."
internet
paige
atrium,"
thief
arse
masochist.
"dios
tequila!"
jalapeno
inspirational
lesbian
astley
you're
pervert
patrick's
obesity!"
tourettes"
skywalker's
"what's
sick."
bubbleÂ
yeasty!"
midget?
crocs"
haven't
you're
take"...is
"you're
dickhead".
âa
âadvice
âhave
âis
âthe
âthe
n)=pa+pn+la+ln.
@doctormoose6
âthe
imaginary
iâm
shampoo
#instagram
#name?
âtaken
denial
pedophile.
iâm
things
111,111,111
r34d
dyslexics
ducked.
lottery.
tweet,
20ãevening
21ãhow
27ãa
pencil.
cigars,
cigars,
me.i
31ãi
35ã

poker.
irrational,
sentence.
sleep
triumph
assisted
die.
luther
joseph.
dessert.
cupcakes
nail
aren't
math
watermelons
aftermath.'
didn't
you.
don't
wifi
pilau
limbo
vowel.
duvet.
pilau
recliner
pissed
pack
hiv?"
i've
i've
can't
penis
donât
balls
what's
whats
doesn't
woman's
words.
menstrual
mental
yvette".
omelette
omelette
you've
juan
you've
erases
can't
youâve
midget
jog.
tyson
pasteurized
losers.
there's
piggy
warfare
can't
don't
isn't
isnt
won't
monotheism
mopeds
happiness.
control-c
mathematicians
jeans
enjoy
climbers
pals
pals
milkman.
spanners
'duh'
9-year-old
sexy
he'd
willows
apathy
sick,
mime,
countdown
mate
hairdresser.
there's
knickers
2500
they're
pervert.
"you're
he's
bossÂ
verb.
boss:ive
boyfriend
he's
didn't
run-d.m.c.,
wank
didn't
he's
calculator
fuming.
overcompensate
recovering
box.
onions
kickboxing
deleted
midget,
coworker
he's
loves
mourning
ballroom"
drag
burying
wasn't
calibre
brigade.
bartender
didn't
rope?"Â
dentist
wife's
wife's
i'll
volcanoes
alzheimer'

what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
what's
you're
whats
whats
whats
whats
whats
whats
whats
whats
whats
whats
whats
whats
whats
whats
clock
yawn,
there's
opens
steals
"i'd
pessimist
they'll
didn't
inventing
aryabhatta
intercom
you're
can't
minefields
israeli
don't
wimbeldon?
"let's
he's
giraffe,
giraffe;
fanta,
dmv.
earry
masseuse
toaster
pub.i'm
mistook
shower
big-game
watches
parents.
evils
say"whoa,
rut.
cookies.
cookies.
imaginasian
smart
didnât
thru
barcode.
exam.
bike,
earwigs
urologist
don't
drive

/Users/geneahn/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [282]:
df_funny[df_funny['text']!=df_funny['revised_text']].count()

text            1747
funny           1747
not_funny       1747
revised_text    1747
dtype: int64

In [283]:
df_funny[df_funny['text']==df_funny['revised_text']].count()

text            3687
funny           3687
not_funny       3687
revised_text    3687
dtype: int64

In [264]:
df_final = pd.concat([df_funny,df_not_funny])

In [266]:
df_final.to_csv('dataset2_crowdtruth_working_updated.csv')

In [267]:
df_final

,text,funny,not_funny
5434,donât knock on deathâs door. hit the doorbell and run. he hates that.,1,0
5435,i discovered i scream the same way whether iâm about to be devoured by a great white shark or if a piece of seaweed touches my foot.,1,0
5436,"my wife used to be a regular customer at mcdonalds. these days, she's more of a large.",1,0
5437,"two can of paint got married, later the bride whispered, ""darling, i think i'm pigment.""",1,0
5438,"- don't be irreplaceable, if you can't be replaced, you can't be promoted.",1,0
5439,- how can you tell a polish woman is on her period? she's only wearing 1 sock,1,0
5440,- what do you call 2 gay guy named bob? oral roberts,1,0
5441,- what do you get when you cross a mexican with an oriental? a car thief who can't drive,1,0
5442,- what do you get when you cross a polack and a mongoloid with one leg? a polaroid 1 step,1,0
5443,- what is better than 4 rose on a piano? two lips on an organ,1,0
